In [1]:
%pip install openai langchain python-dotenv langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()  # Cargar archivo .env
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("No se encontró la clave OPENAI_API_KEY en el archivo .env")

# Crear cliente LangChain con OpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
print("Cliente LangChain con OpenAI inicializado correctamente.")

Cliente LangChain con OpenAI inicializado correctamente.


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize the LLM (uses your OpenAI API key from environment)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

# Create a simple prompt
prompt = ChatPromptTemplate.from_template(
    "Explica en dos frases el concepto de {tema}."
)

# Combine the components using LCEL (LangChain Expression Language)
chain = prompt | llm | StrOutputParser()

# Run it
result = chain.invoke({"tema": "aprendizaje automático"})
print(result)

El aprendizaje automático es una rama de la inteligencia artificial que permite a las computadoras aprender y mejorar su rendimiento en tareas específicas a partir de datos, sin ser programadas explícitamente para ello. Utiliza algoritmos para identificar patrones y hacer predicciones o decisiones basadas en la información proporcionada.


In [ ]:
from langchain.chains import SimpleSequentialChain

# Primer paso: obtener una descripción
primer_prompt = PromptTemplate(
    input_variables=["tema"],
    template="Explica brevemente el concepto de {tema}."
)
primer_chain = LLMChain(llm=llm, prompt=primer_prompt)

# Segundo paso: generar aplicación educativa
segundo_prompt = PromptTemplate(
    input_variables=["concepto"],
    template="Propón una aplicación educativa del siguiente concepto: {concepto}."
)
segundo_chain = LLMChain(llm=llm, prompt=segundo_prompt)

# Combinar ambos pasos en una cadena secuencial
chain_secuencial = SimpleSequentialChain(chains=[primer_chain, segundo_chain])

# Ejecutar la cadena completa
resultado = chain_secuencial.run("realidad aumentada")
print(resultado)


In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

# 1) LLM (usa tu OPENAI_API_KEY en el entorno)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
to_str = StrOutputParser()

# 2) Paso 1: explicar brevemente el concepto de {tema}
primer_prompt = ChatPromptTemplate.from_template(
    "Explica brevemente el concepto de {tema}."
)
primer_paso = primer_prompt | llm | to_str
# `primer_paso` produce un string, por ejemplo: "La realidad aumentada es ..."

# 3) Paso 2: proponer una aplicación educativa usando la salida del paso 1 como {concepto}
segundo_prompt = ChatPromptTemplate.from_template(
    "Propón una aplicación educativa del siguiente concepto: {concepto}."
)
segundo_paso = segundo_prompt | llm | to_str

# 4) Encadenar: mapear la entrada {tema} al primer paso, y su salida a {concepto} del segundo
cadena_secuencial = {"concepto": primer_paso} | segundo_paso

# 5) Ejecutar la cadena completa
resultado = cadena_secuencial.invoke({"tema": "realidad aumentada"})
print(resultado)

**Aplicación Educativa: "Aula Aumentada"**

**Descripción General:**
"Aula Aumentada" es una aplicación educativa diseñada para enriquecer el aprendizaje en diversas materias a través de la realidad aumentada. La aplicación permite a los estudiantes interactuar con contenidos digitales relacionados con su entorno físico, facilitando una experiencia de aprendizaje más inmersiva y contextualizada.

**Características de la Aplicación:**

1. **Contenido Interactivo:**
   - Los docentes pueden crear y cargar contenido educativo en la plataforma, como imágenes, videos, modelos 3D y cuestionarios interactivos que se superponen en el entorno real.
   - Por ejemplo, en una clase de biología, los estudiantes pueden apuntar su dispositivo hacia una planta y ver información sobre su estructura celular, su ecosistema y su ciclo de vida.

2. **Exploración del Entorno:**
   - La aplicación utiliza la cámara del dispositivo para reconocer objetos y lugares. Al escanear un objeto específico, los estudi

In [15]:
# Instalar si hace falta:
# %pip install -U langchain langchain-openai

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser

# LLM (requiere OPENAI_API_KEY en el entorno)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
to_str = StrOutputParser()

# Prompt con hueco para el historial
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente educativo claro y conciso."),
    MessagesPlaceholder("chat_history"),      # ← aquí va la memoria
    ("human", "{input}")
])

# Cadena base
chain = prompt | llm | to_str

# Memoria simple como lista de mensajes
history: list = []

def chat(user_text: str) -> str:
    """
    Envía un turno del usuario, usa el historial y actualiza la memoria
    con el par (usuario, asistente).
    """
    global history
    # Ejecutar la cadena inyectando el historial actual
    answer = chain.invoke({"input": user_text, "chat_history": history})
    # Actualizar memoria (guardar los dos mensajes)
    history += [HumanMessage(content=user_text), AIMessage(content=answer)]
    return answer

# --- Ejemplo de uso (tres turnos) ---
print(chat("Hola, soy un profesor de informática."))
print(chat("¿Puedes explicarme cómo introducir IA a mis estudiantes?"))
print(chat("¿Qué ejemplos prácticos puedo usar en la clase?"))



¡Hola! ¿En qué puedo ayudarte hoy con respecto a la enseñanza de informática?
Claro, aquí tienes algunos pasos para introducir la inteligencia artificial (IA) a tus estudiantes:

1. **Conceptos Básicos**:
   - Comienza explicando qué es la IA: la capacidad de una máquina para imitar la inteligencia humana.
   - Presenta términos clave como aprendizaje automático, redes neuronales y procesamiento del lenguaje natural.

2. **Historia y Aplicaciones**:
   - Proporciona una breve historia de la IA y sus hitos importantes.
   - Muestra aplicaciones prácticas en la vida cotidiana, como asistentes virtuales (Siri, Alexa), recomendaciones de productos y reconocimiento facial.

3. **Demostraciones Prácticas**:
   - Usa herramientas y plataformas accesibles como Google Teachable Machine o Scratch para mostrar cómo funciona el aprendizaje automático.
   - Realiza proyectos simples donde los estudiantes puedan crear modelos básicos de IA.

4. **Ética y Consideraciones**:
   - Discute los aspectos 